# Information Retrieval
***
## | Latent Semantic Indexing |

Damien Draime - Apr 19

### Introduction

Typically, information is retrieved by literally matching terms in documents with those of a query. However, **lexical matching methods** can be inaccurate when they are used to match a user’s query. Since there are usually many ways to express a given concept (synonymy), the literal terms in a user’s query may not match those of a relevant document. In addition, most words have multiple meanings (polysemy), so terms in a user’s query will literally match terms in irrelevant documents. A better approach would allow users to retrieve information on the basis of a conceptual topic or meaning of a document.<br>
**Latent Semantic Indexing** (**LSI**) tries to overcome the problems of lexical matching by using statistically derived conceptual indices instead of individual words for retrieval. A truncated singular value decomposition (SVD) is used to estimate the structure in word usage across documents. Retrieval is performed using the database of singular values and vectors obtained from the truncated SVD.<br><br>
 - [Motivation](#Motivation)
 - [LSI and Linear Algebra](#LSI-and-Linear-Algebra)
 - [LSI for retrieval](#LSI-for-retrieval)
     - [Variant 01 - Ak](#Variant-01---Ak)
     - [Variant 02 - Vk](#Variant-02---Vk)
     - [Variant 03 - Expand](#Variant-03---Expand)


***

*Based on __[Deerwester et al. - Indexing by Latent Semantic Analysis](http://lsa.colorado.edu/papers/JASIS.lsi.90.pdf)__, __[Uni-Freiburg's Information Retrieval - Lecture 10](https://daphne.informatik.uni-freiburg.de/ws1617/InformationRetrieval/svn-public/public/slides/lecture-10.pdf)__, and __[KU Leuven's Text-Based Information Retrieval - Lecture 04](https://p.cygnus.cc.kuleuven.be/bbcswebdav/pid-22306777-dt-content-rid-175870631_2/courses/B-KUL-H02C8a-1819/Chapter3-AdvancedRepresentations-Part1.pdf)__.*<br>
*Other sources: __[Stanford's NLP website](https://nlp.stanford.edu/IR-book/html/htmledition/latent-semantic-indexing-1.html)__.*

### Motivation
Let's look at a toy example made of a collection of D documents and a vocabulary of W terms. For this example, D=6, and W=4.

In [6]:
import numpy as np
import pandas as pd
A = pd.read_csv('dataset/toy_example.csv',index_col=0)
A

,document 1,document 2,document 3,document 4,document 5,document 6
index,,,,,,
internet,1,1,0,1,0,0
web,1,0,1,1,0,0
surfing,1,1,1,2,1,1
beach,0,0,0,1,1,1


This is thus a matrix representation of the documents and the terms that compose each of them. We will refer to this matrix as __A__<sub>m x n</sub>. Each column refers to a specific document, each row to a term. A<sub>ij</sub> indicates the frequency of the term i in the document j (*tf<sub>ij</sub>*).<br>
*E.g.: the term `surfing` appears in all documents once, except for `document 4` where it appears twice.*<br><br>
Intuitively, we notice that the first 3 documents are related to "surfing the web", while the last 2 are about "beach/surfing waves". Hence we can identify that there are two concepts. *`document 4` is a mix of the two concepts.*<br><br>
Let's assume that we have a query: "web surfing", represented as a vector **_q_**: $[0 1 1 0]$. In order to retrieve the relevant documents for this query, we use a **lexical matching method** by doing a dot product A<sub>j</sub> q. This gives us a similarity score between the query and the document j. <br>(*Note that this similarity score is based on term frequencies and thus has a bias towards long documents. This will be ignored here but one might want to first normalize the tf<sub>ij</sub>.*)


In [7]:
q = np.array([0, 1, 1, 0])
for document in A:
    print (document, 'has a relevance score of', A[document].dot(q))

document 1 has a relevance score of 2
document 2 has a relevance score of 1
document 3 has a relevance score of 2
document 4 has a relevance score of 3
document 5 has a relevance score of 1
document 6 has a relevance score of 1


We could then use this relevance scores to rank the documents and thus display to the user only the n<sup>th</sup> first documents that we consider relevant for his/her query. However, you will quickly notice a problem with this approach. `document 2` should be considered as relevant since it is also related to "surfing the web". But since the term `web` from the query does not match the term `internet` from the document, the relevance score dropped. Thus, althought `document 2` and `document 3` should intuitively have the same relevance, `document 2` has a lower similarity score because our approach only consider exact match.<br>
Actually, `internet` and `web` are *synonyms* but this notion is not integrated in the lexical matching approach. Another problem can arise with *polysems* such as `surfing`: its meaning depends on the context.<br><br>Those are not benign problems and occur more frequently than we think. Consider the following set of terms: *bank, finance, river, stream, money.*<br><br>
What if we adjusted A so that we take into account the synonyms ? Let's see:

In [8]:
A_adjusted = A.copy()
for document in A_adjusted:
    if A_adjusted[document]['internet'] == 1 or A_adjusted[document]['web'] == 1:
        A_adjusted[document]['internet'] = 1
        A_adjusted[document]['web'] = 1
A_adjusted

,document 1,document 2,document 3,document 4,document 5,document 6
index,,,,,,
internet,1,1,1,1,0,0
web,1,1,1,1,0,0
surfing,1,1,1,2,1,1
beach,0,0,0,1,1,1


In [9]:
for document in A_adjusted:
    print (document, 'has a relevance score of', A_adjusted[document].dot(q))

document 1 has a relevance score of 2
document 2 has a relevance score of 2
document 3 has a relevance score of 2
document 4 has a relevance score of 3
document 5 has a relevance score of 1
document 6 has a relevance score of 1


Now the similarity score for `document 2` and `document 3` are the same. This seems better, right ? Well, this is the idea behind **Latent Semantic Indexing**.<br>
Latent Semantic Indexing represents queries and document with vectors by mapping the term vectors of documents and query into a *low dimensionsal space* which is associated with statistical **concepts**. This would allow the retrieval of documents even when they are not indexed by the query index terms. The method used to reduce the dimensionality is the **Singular Value Decomposition**.

### LSI and Linear Algebra

First, it is interesting to note that the new matrix `A_adjusted` has become a matrix of **rank** 2. (The original matrix A had a rank 4). this means that we can write each column/row as a linear combination of the same two base columns/rows. In our case the 2 base columns are `document 1`, and `document 5`. Respectively, the 2 base rows are `internet`, and `beach`.<br><br>
*The rank of a matrix A corresponds to the maximal number of linearly independent columns of A. colomn rank = row rank. The rank of an m x n matrix is nonnegative integer and cannot be greater than either m or n:* $rank(A_{m x n}) \leq min(m,n)$<br><br>
It is important to notice that we can thus derive A_adjusted from those base columns and rows:

In [10]:
B = A_adjusted[['document 1','document 5']]
V = A_adjusted.loc[['internet','beach']]
print(np.dot(B,V)) # returns again the original matrix A_adjsuted

[[1 1 1 1 0 0]
 [1 1 1 1 0 0]
 [1 1 1 2 1 1]
 [0 0 0 1 1 1]]


The columns from the matrix B represent **underlying concepts** present in the collection of documents. And the columns in the matrix V are the representation of documents in the **concept/latent space**. It is crucial to note that this concept space is of a lower dimension (here 2) than original matrix A's dimension.<br>
Note also that a column vector j from matrix V represents the linear combination for the two concepts (from B) for the document j. 

Hence the goal of LSI is to find a matrix __A<sub>k</sub>__ of lower rank than the original matrix __A__. Formally, given an m x n term-document matrix A, find a matrix A<sub>k</sub> of rank **k** such that k < rank(A) and the difference between A and A<sub>k</sub> is as small as possible.<br>
$A_{k} = argmin_{A_{k} m x n\;with\;rank\;k} ||A - A_{k}||$<br><br>
This *difference* between the two matrices can be computed with the Frobenius-norm:<br>
$||A - A_{k}|| = \sqrt{\sum_{ij} (A_{ij} - A_{k ij})^{2}}$
<br><br>
*Note that A<sub>k</sub> will still have the same dimension than A*.<br>


The question thus becomes: how do we find this A<sub>k</sub> ?<br>
For this we can use the so-called **Singular Value Decomposition** (**SVD**) of the matrix A. SVD is based on the theorem that for any m x n matrix A of rank r, there exist matrices **U, D, and V**, such that
$A = U . D . V$<br>
_where<br>
 -U is an m x r matrix with pairwise orthogonal columns,<br>
 -D is an r x r matrix with non-zero entries only on its diagonal,<br>
 -V is an r x n matrix with pairwise orthogonal rows._
<br><br>
In numpy, it is very easy to find such decomposition for a matrix:

In [11]:
np.set_printoptions(formatter = {"float": lambda x: ("%5.2f" % x)})
U, D, V = np.linalg.svd(A)
D = np.diag(D)
V = V[:4,:]
print ('U is a', U.shape, 'matrix:\n', U, '\n')
print ('D is a', D.shape, 'diagonal matrix:\n', D, '\n')
print ('V is a', V.shape, 'matrix:\n', V)

U is a (4, 4) matrix:
 [[-0.37 -0.47  0.71 -0.38]
 [-0.37 -0.47 -0.71 -0.38]
 [-0.78  0.12  0.00  0.61]
 [-0.34  0.74  0.00 -0.58]] 

D is a (4, 4) diagonal matrix:
 [[ 3.80  0.00  0.00  0.00]
 [ 0.00  1.55  0.00  0.00]
 [ 0.00  0.00  1.00  0.00]
 [ 0.00  0.00  0.00  0.38]] 

V is a (4, 6) matrix:
 [[-0.40 -0.30 -0.30 -0.69 -0.30 -0.30]
 [-0.53 -0.22 -0.22  0.03  0.56  0.56]
 [ 0.00  0.71 -0.71 -0.00  0.00  0.00]
 [-0.40  0.60  0.60 -0.34  0.06  0.06]]


We can now verify the theorem:

In [12]:
print(U.dot(D).dot(V).astype(int)) #returns the original matrix A

[[1 1 0 1 0 0]
 [1 0 1 1 0 0]
 [1 1 1 2 1 1]
 [0 0 0 1 1 1]]


A<sub>k</sub> can now be easily computed thanks to this SVD because, given a k, we have:<br>
$A_k = U_k . D_k . V_k$<br>
_where<br>
 -U<sub>k</sub> is an m x k matrix consiting of the first k columns of U,<br>
 -D<sub>k</sub> is an k x k matrix consisting of the upper k x k part of D,<br>
 -V<sub>k</sub> is an k x n matrix consisting of the first k rows of V._
 <br>![figure 01 - Ak](img/UDV.PNG)<br>
 *k* is a parameter _provided by the user_. It is related to the number of concepts that are present in the collection of documents. In our example, we can set it to 2 because we intuitively see that there are two main concepts. It is with this value that we will reduce the vector space. As such, k < W and k < D. In reality it is not always simple to determine the right value for k. <br>
 Let's now find this m x n matrix A<sub>k</sub> of rank k that minimizes $||A - A_{k}||$:

In [14]:
k = 2
Uk = U[:,:k]
Dk = D[:k,:k]
Vk = V[:k,:]
Ak = Uk.dot(Dk).dot(Vk)
Ak = pd.DataFrame(
    data = Ak, 
    index = ['internet','web','surfing','beach'], 
    columns = ['document 1','document 2','document 3','document 4','document 5','document 6']
)
Ak

,document 1,document 2,document 3,document 4,document 5,document 6
internet,0.942024,0.586451,0.586451,0.950631,0.008607,0.008607
web,0.942024,0.586451,0.586451,0.950631,0.008607,0.008607
surfing,1.092679,0.861802,0.861802,2.078920,0.986242,0.986242
beach,-0.089224,0.133047,0.133047,0.924022,1.013246,1.013246


Doesn't it look to what the A_adjusted matrix we manually built in the section above ? A_k represents A as in a lower k dimensional space (rank k) in such a way that the representations in the original space are changed as little as possible when measured by the Frobenius-norm. <br>
Let's now recompute the similarity score between the query **q** and __A<sub>k</sub>__ to see how performs LSI:

In [9]:
for document in Ak:
    print (document, 'has a relevance score of', round(Ak[document].dot(q),2))

document 1 has a relevance score of 2.03
document 2 has a relevance score of 1.45
document 3 has a relevance score of 1.45
document 4 has a relevance score of 3.03
document 5 has a relevance score of 0.99
document 6 has a relevance score of 0.99


Note that, instead of the **Inner product similarity** we have used so far, we could also have decided to use the **Cosine similarity**. This defines the similarity as the cosine of the angle between query and document vectors:
$sim(d_j,q) = \frac{d_j^t . q}{||d_j|| ||q||}$:

In [35]:
from scipy import spatial
for document in Ak:
    print (document, 'has a relevance score of', 1 - round(spatial.distance.cosine(Ak[document],q),2))

document 1 has a relevance score of 0.83
document 2 has a relevance score of 0.85
document 3 has a relevance score of 0.85
document 4 has a relevance score of 0.81
document 5 has a relevance score of 0.5
document 6 has a relevance score of 0.5


### LSI for retrieval

#### Variant 01 - Ak

In this variant, we use the matrix of lower rank that we defined earlier: **A<sub>k</sub>**. Recall that this matrix is also the matrix of rank k that is the closest to A. <br>
However, this matrix A<sub>k</sub> is a dense matrix (i.e.: most of the entries are non-zero). This poses problem when it comes to store it and compute similarity.

In [16]:
Ak

,document 1,document 2,document 3,document 4,document 5,document 6
internet,0.942024,0.586451,0.586451,0.950631,0.008607,0.008607
web,0.942024,0.586451,0.586451,0.950631,0.008607,0.008607
surfing,1.092679,0.861802,0.861802,2.078920,0.986242,0.986242
beach,-0.089224,0.133047,0.133047,0.924022,1.013246,1.013246


#### Variant 02 - Vk

Here we rather use the **V<sub>k</sub>** matrix which gives a representation of the documents in the k-dimensional concept space. This matrix will thus have as many rows as the concepts k defined by the user.<br>
This variant has the advantage that it will be easier to query and store since the dimention will be k x D.

In [20]:
Vk = pd.DataFrame(
    data = Vk, 
    index = ['concept 1','concept 2'], 
    columns = ['document 1','document 2','document 3','document 4','document 5','document 6']
)
Vk

,document 1,document 2,document 3,document 4,document 5,document 6
concept 1,-0.399508,-0.302930,-0.302930,-0.694691,-0.295182,-0.295182
concept 2,-0.528702,-0.224807,-0.224807,0.027466,0.556167,0.556167


The drawback is that one has to convert the query __q__ to this **concept space** though.<br>
We know that:
$$V_k^t = A_k^t . U_k . D_k^{-1}$$<br>
Thus if we want to map a query q to this new space, we have to apply:
$$q_k^t = q^t . U_k . D_k^{-1}$$

In [28]:
qk = q.transpose().dot(Uk).dot(np.linalg.inv(Dk))
qk

array([-0.30, -0.22])

In [36]:
for document in Vk:
    print (document, 'has a relevance score of', 1 - round(spatial.distance.cosine(Ak[document],q),2))

document 1 has a relevance score of 0.83
document 2 has a relevance score of 0.85
document 3 has a relevance score of 0.85
document 4 has a relevance score of 0.81
document 5 has a relevance score of 0.5
document 6 has a relevance score of 0.5


If we want to map also a new document into this concept space, we would have to apply the same transformation. Imagine that the new document `document 7` is $[2 1 3 0]$. Once mapped in the concept space it will be:

In [33]:
document_7 = np.array([2,1,3,0])
document_7k = document_7.transpose().dot(Uk).dot(np.linalg.inv(Dk))
document_7k

array([-0.91, -0.67])

This technique of adding a new document to the new representation is called **folding in**. With this technique we can add new documents to the collection and map them to the concept space, as they come. However, since the LSI is not recomputed with those new documents, slowly the quality of the current representation will be degraded. At some point, it will be necessary to perform again a LSI to capture the concepts and information from the new documents.

#### Variant 03 - Expand

Remember what we did in the first section ? We manually adjusted the matrix A bused on our gut feelings about the underlying concepts present in the collection of documents. This allowed us to have a sparse matrix **A_adjusted**. <br>
We can define a matrix **T<sub>k</sub>** that allows us to do this conversion for each document:

In [29]:
Tk = pd.DataFrame(
    data = [[1,1,0,0],[1,1,0,0],[0,0,1,0],[0,0,0,1]], 
    index = ['internet','web','surfing','beach'], 
    columns = ['internet','web','surfing','beach']
)
Tk

,internet,web,surfing,beach
internet,1,1,0,0
web,1,1,0,0
surfing,0,0,1,0
beach,0,0,0,1


This matrix T is then used to expand the documents. If a document has `internet` then we expand it with `web`. If a document has `web`, we expand it with `internet`. <br>
Here is an example with `document 2`:

In [34]:
A['document 2'].dot(Tk)

internet    1
web         1
surfing     1
beach       0
Name: document 2, dtype: int64

This trick is useful because we can now get a sparse matrix again. However, the question is "How to get the matrix T in the first place?"

In summary, the Latent Semantic Indexing claims that:
- semantic information can be derived from a word-document co-occurrence matrix
- dimensionality reduction is an essential part of this derivation
- words and documents can be represented as points in the Euclidean space.
***